# Breaking into error mitigation with Mitiq's calibration module

Things that this tutorial covers:
- Getting started with Mitiq's calibration module  
- Use Qiskit noisy simulatro with FakeJakarta
- Run calibration with RBSettings  
- Conclusion

## Getting started with Mitiq

In [1]:
import numpy as np 
from mitiq import MeasurementResult
from mitiq.interface.mitiq_qiskit.conversions import to_qiskit, from_qiskit
import mitiq

In [2]:
from mitiq.benchmarks import generate_rb_circuits
from mitiq.zne import execute_with_zne
from mitiq.zne.inference import LinearFactory, RichardsonFactory
from mitiq.zne.scaling import (
    fold_gates_at_random,
    fold_global,
)

from mitiq.calibration import Calibrator, ZNESettings, execute_with_mitigation
from mitiq.calibration.settings import Settings

In [3]:
from qiskit.providers.fake_provider import FakeJakarta  # Fake (simulated) QPU

### Define the circuit to study

#### Global variables
Define global variables for the quantum circuit of interest: number of qubits, depth of the quantum circuit and number of shots.

In [4]:
n_qubits = 2
depth_circuit= 20
shots = 10 ** 3

#### Quantum circuit: Randomized benchmarking (RB)
We now use Mitiq's built-in `generate_rb_circuits` from the `mitiq.benchmarks` module to define the quantum circuit.

In [5]:
circuit = generate_rb_circuits(n_qubits, depth_circuit,return_type="qiskit")[0]#,trials=3)#[0]
circuit.measure_all()
print(len(circuit))
print(circuit)

218
        ┌──────────┐ ┌────┐ ┌─────────┐   ┌───────┐┌───┐    ┌────┐            »
   q_0: ┤ Ry(-π/2) ├─┤ √X ├─┤ Ry(π/2) ├─■─┤ Rx(0) ├┤ Y ├────┤ √X ├──────────■─»
        ├──────────┤┌┴────┴┐└─────────┘ │ └─┬───┬─┘├───┴┐┌──┴────┴──┐┌────┐ │ »
   q_1: ┤ Ry(-π/2) ├┤ √Xdg ├────────────■───┤ Y ├──┤ √X ├┤ Ry(-π/2) ├┤ √X ├─■─»
        └──────────┘└──────┘                └───┘  └────┘└──────────┘└────┘   »
meas: 2/══════════════════════════════════════════════════════════════════════»
                                                                              »
«         ┌───────┐    ┌───┐   ┌────┐   ┌─────────┐   ┌───┐    ┌────┐  »
«   q_0: ─┤ Rx(0) ├────┤ Y ├───┤ √X ├─■─┤ Ry(π/2) ├─■─┤ Y ├────┤ √X ├──»
«        ┌┴───────┴┐┌──┴───┴──┐├────┤ │ └─┬──────┬┘ │ ├───┴┐┌──┴────┴─┐»
«   q_1: ┤ Ry(π/2) ├┤ Ry(π/2) ├┤ √X ├─■───┤ √Xdg ├──■─┤ √X ├┤ Ry(π/2) ├»
«        └─────────┘└─────────┘└────┘     └──────┘    └────┘└─────────┘»
«meas: 2/══════════════════════════════════════════════════════════════

We define a function that executes the quantum circuits and returns the expectation value. This is consumed by Mitiq's `execute_with_zne`. 

In [6]:
def execute_circuit(circuit):
    """Execute the input circuit and return the expectation value of |00..0><00..0|"""
    noisy_backend = FakeJakarta()
    noisy_result = noisy_backend.run(circuit, shots=shots).result()
    noisy_counts = noisy_result.get_counts(circuit)
    noisy_expectation_value = noisy_counts[n_qubits * "0"] / shots
    return noisy_expectation_value

In [7]:
mitigated = execute_with_zne(circuit, execute_circuit)
unmitigated = execute_circuit(circuit)
ideal = 1 #property of RB circuits 

print("ideal = \t \t",ideal)
print("unmitigated = \t \t",unmitigated)
print("mitigated = \t \t",mitigated)

ideal = 	 	 1
unmitigated = 	 	 0.95
mitigated = 	 	 0.9699999999999993


## Using calibration to improve the results

 Let's consider as executor a noisy quantum circuit using Qiskit noisy backend simulators, `FakeJakarta`. Right now the calibration module does not natively support Qiskit circuits, so in the executor, we use Mitiq's conversion functions to convert the Qiskit circuit with `mitiq.interface.mitiq_qiskit.conversions.to_qiskit`. 

In [8]:
def execute_calibration(cirq_circuit):
    """Execute the input circuits and return the measurement results."""
    qiskit_circuit = to_qiskit(cirq_circuit)
    noisy_backend = FakeJakarta()
    noisy_result = noisy_backend.run(qiskit_circuit, shots=shots).result()
    noisy_counts = noisy_result.get_counts(qiskit_circuit)
    measurements = MeasurementResult.from_counts(noisy_counts)
    return measurements

We import from the calibration module the key ingredients to use `mitiq.calibration`: the `Calibrator` class, the `mitiq.calibration.settings.Settings` class and the `execute_with_mitigation` function. 

Currently `mitiq.calibration` supports ZNE as a technique to calibrate from, tuning different scale factors, extrapolation methods and circuit scaling methods. 

Let's run the calibration using an ad-hoc RBSettings and using the `log=True` option in order to print the list of experiments run. 

- benchmarks: Circuit type: "rb"
- strategies: use various "zne" strategies, testing various "scale_noise" methods, and ZNE factories for extrapolation


In [9]:
RBSettings = Settings(
    benchmarks=[
        {
            "circuit_type": "rb",
            "num_qubits": n_qubits,
            "circuit_depth": depth_circuit,
        },
    ],
    strategies=[
        {
            "technique": "zne",
            "scale_noise": fold_global,
            "factory": RichardsonFactory([1.0, 2.0, 3.0]),
        },
        {
            "technique": "zne",
            "scale_noise": fold_global,
            "factory": RichardsonFactory([1.0, 3.0, 5.0]),
        },
        {
            "technique": "zne",
            "scale_noise": fold_global,
            "factory": LinearFactory([1.0, 2.0, 3.0]),
        },
        {
            "technique": "zne",
            "scale_noise": fold_global,
            "factory": LinearFactory([1.0, 3.0, 5.0]),
        },
        {
            "technique": "zne",
            "scale_noise": fold_gates_at_random,
            "factory": RichardsonFactory([1.0, 2.0, 3.0]),
        },
        {
            "technique": "zne",
            "scale_noise": fold_gates_at_random,
            "factory": RichardsonFactory([1.0, 3.0, 5.0]),
        },
        {
            "technique": "zne",
            "scale_noise": fold_gates_at_random,
            "factory": LinearFactory([1.0, 2.0, 3.0]),
        },
        {
            "technique": "zne",
            "scale_noise": fold_gates_at_random,
            "factory": LinearFactory([1.0, 3.0, 5.0]),
        },
    ],
)

In [10]:
cal = Calibrator(execute_calibration, RBSettings)
print(cal.get_cost())
cal.run(log=True)

{'noisy_executions': 8, 'ideal_executions': 0}
Ran rb circuit using: ['ZNE', 'RichardsonFactory', [1.0, 2.0, 3.0], 'fold_global']
✅ ideal: 1.00	noisy: 0.94	mitigated: 1.02
Ran rb circuit using: ['ZNE', 'RichardsonFactory', [1.0, 3.0, 5.0], 'fold_global']
✅ ideal: 1.00	noisy: 0.94	mitigated: 0.98
Ran rb circuit using: ['ZNE', 'LinearFactory', [1.0, 2.0, 3.0], 'fold_global']
✅ ideal: 1.00	noisy: 0.94	mitigated: 0.98
Ran rb circuit using: ['ZNE', 'LinearFactory', [1.0, 3.0, 5.0], 'fold_global']
✅ ideal: 1.00	noisy: 0.94	mitigated: 0.97
Ran rb circuit using: ['ZNE', 'RichardsonFactory', [1.0, 2.0, 3.0], 'fold_gates_at_random']
✅ ideal: 1.00	noisy: 0.94	mitigated: 0.94
Ran rb circuit using: ['ZNE', 'RichardsonFactory', [1.0, 3.0, 5.0], 'fold_gates_at_random']
✅ ideal: 1.00	noisy: 0.94	mitigated: 0.99
Ran rb circuit using: ['ZNE', 'LinearFactory', [1.0, 2.0, 3.0], 'fold_gates_at_random']
✅ ideal: 1.00	noisy: 0.94	mitigated: 0.97
Ran rb circuit using: ['ZNE', 'LinearFactory', [1.0, 3.0, 5.0],

As you can see above, several experiments were run, and each one has either a red cross (❌) or a green check (✅) to signal whether the error mitigation experiment obtained an expectation value that is better than the non-mitigated one. 

In [11]:
calibrated_mitigated=execute_with_mitigation(circuit, execute_circuit, calibrator=cal)
mitigated=execute_with_zne(circuit, execute_circuit)
unmitigated=execute_circuit(circuit)

print("ideal = \t \t",ideal)
print("unmitigated = \t \t",unmitigated)
print("mitigated = \t \t",mitigated)
print("calibrated_mitigated = \t",calibrated_mitigated)


ideal = 	 	 1
unmitigated = 	 	 0.96
mitigated = 	 	 0.9699999999999982
calibrated_mitigated = 	 0.9843749999999991
